In [2]:
import pandas as pd
import numpy as np
import nltk
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os, glob
import csv
from functools import reduce
import re
import string


In [11]:
labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]
doc_names = ["fox_news", "valkyrae", "terror", "usanews", "nogla", "pewdiepie"]
src_folder = "tfidf_scores_results_per_video/"
src_folder2 = "correlations_of_tfidf_scores_per_label/"
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

dest_folder = "0_FOR_ANALYSIS_RESULTS/"
# ("fox_news", "foxnews_FULL.csv"), ("Valkyrae", "rae_FULL.csv"),("Terror", "terror_FULL.csv"), 
#          ("usa_news", "usanews.csv"), ("nogla", "nogla0.csv"), ("pewdiepie", "pew0.csv"),
files = [ ("gaming channels","gaming_channels_GENRE.csv" ),
         ("news","news_channel_GENRE.csv" )]
prefix_s = "spr_"
prefix_p = "pval_"


# TOP BI-GRAMS BASED ON TFIDF SCORES

In [18]:
src_folder = "tfidf_scores_WORDS_per_video/"
dfs = []
for fn, file in files:
    vals = {}
    df = pd.read_csv(src_folder+file, index_col = 0)
    features = df.columns.difference(labels)
    for label in labels:
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(20)
        dl.index.name = fn
        dl = dl.rename({dl.columns[0]: label}, axis='columns')
        
        display(dl)
        
        dfs.append(dl)
        #$dl.to_csv(dest_folder+"TOP_TFIDF_"+file)

,Direct Harassment
gaming channels,
stream snipe,1.000000
die lmao,1.000000
holy shit,1.000000
nogla yell,1.000000
luck brian,1.000000
nancy cringe,1.000000
sooo dirty,1.000000
suck bad,1.000000
suck imposter,1.000000


,Hate Speech
gaming channels,
blm joke,8.100984
death blm,4.000000
black guy,3.498167
blm racist,3.000000
lives matter,2.275872
blm suck,2.000000
guy nogla 2020,1.444436
guy nogla,1.411613
death black,1.397464


,Sexual Harassment
gaming channels,
ban nogla,1.0
live matter,1.0
felix gay,1.0
00 rae,0.0
ppl happier totally,0.0
ppl delay,0.0
ppl dislike,0.0
ppl dislike vid,0.0
ppl enjoy,0.0


,Trolling
gaming channels,
members money,5.366563
donations charity members,5.366563
donations charity,5.366563
charity members money,5.366563
charity members,5.366563
blm terrorist,1.483221
gon na,1.390565
purge 2021,1.000000
jack just,1.000000


,Others
gaming channels,
play guy,1.000000
kill nogla,1.000000
just leave,0.707107
guy lol,0.707107
ppl idk,0.000000
ppl dislike vid,0.000000
ppl enjoy,0.000000
ppl enjoy content,0.000000
ppl happier,0.000000


...

,Hate Speech
news,
china joe,12.111777
vote trump,9.721189
biden suck,9.156355
vote biden,7.336646
biden prison,6.764711
joe biden,6.145703
china virus,5.164318
moron biden,4.705372
lady gaga,4.581937


,Sexual Harassment
news,
lady gag,3.000000
lady gaga,1.479019
shitmer husband,1.000000
kamala harris,1.000000
commie bitch,1.000000
gaga make gag,0.899710
gag lady,0.899710
make gag,0.899710
make gag lady,0.899710


,Trolling
news,
joe biden,10.574289
china joe,3.709680
biden prison,3.675389
vote trump,3.592486
biden win,3.055355
biden china,3.000000
moron biden,2.852686
fake news,2.794457
vote biden,2.788979


,Others
news,
traitor hang,3.747841
sunny yu,1.742386
beijing biden,1.571397
civil war,1.502689
biden voter,1.467111
life matter,1.397271
terror attack,1.071269
canada trump,1.000000
face mask,1.000000


,Toxic
news,
biden suck,11.156355
joe biden,8.200592
lady gaga,5.034498
cnn suck,4.000000
vote biden,3.657966
moron biden,3.470248
biden harris,3.329156
vote trump,2.963712
china joe,2.883895


[                   Direct Harassment
 gaming channels                     
 stream snipe                1.000000
 die lmao                    1.000000
 holy shit                   1.000000
 nogla yell                  1.000000
 luck brian                  1.000000
 nancy cringe                1.000000
 sooo dirty                  1.000000
 suck bad                    1.000000
 suck imposter               1.000000
 type relationship           1.000000
 speedy suck                 1.000000
 smell like                  1.000000
 tom castillo                1.000000
 way jack                    0.770844
 want kill                   0.742201
 game watch                  0.721852
 time die                    0.717240
 poor kid talk               0.707107
 poor people                 0.707107
 poor kid                    0.707107,
                  Hate Speech
 gaming channels             
 blm joke            8.100984
 death blm           4.000000
 black guy           3.498167
 blm racist  

# TOP BI-GRAMS BASED ON LOWEST P-VALUE

In [33]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder = "corr_words_per_label/"

for file in genre_files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    df.dropna(axis = 0, how = 'all', inplace = True)
    for label in labels: 
        dp = df[(df[label + " p-val"] > 0)].sort_values(by=[label + " p-val"], ascending = True).head(5)
        top_features = list(dp[label + " p-val"].index)
        dt = dp[ [label + " p-val", label + " corr coeff"]].T[top_features]
        display(dt.T)

,Direct Harassment p-val,Direct Harassment corr coeff
ca nt,0.070484,0.774597
play game terrible,0.177808,0.632456
play game,0.177808,0.632456
poor kid,0.177808,0.632456
stream snipe,0.177808,0.632456


,Hate Speech p-val,Hate Speech corr coeff
lives matter,0.070484,0.774597
death black,0.070484,0.774597
life matter,0.070484,0.774597
guy lol,0.070484,0.774597
blm suck,0.070484,0.774597


,Sexual Harassment p-val,Sexual Harassment corr coeff
ban nogla,0.177808,0.632456
matter george floyd,0.541470,-0.316228
black lives matter,0.541470,-0.316228
black lives,0.541470,-0.316228
matter george,0.541470,-0.316228


,Trolling p-val,Trolling corr coeff
blm terrorist,0.070484,0.774597
ban nogla,0.177808,0.632456
black life matter,0.353057,0.464758
ca nt,0.353057,0.464758
life matter,0.353057,0.464758


,Others p-val,Others corr coeff
play guy,0.177808,0.632456
guy lol,0.353057,0.464758
black lives matter,0.541470,-0.316228
black lives,0.541470,-0.316228
matter george,0.541470,-0.316228


...

,Hate Speech p-val,Hate Speech corr coeff
say indianapolis,0.070484,0.774597
new world,0.070484,0.774597
new world order,0.070484,0.774597
night terrifying,0.070484,0.774597
dems ruin,0.070484,0.774597


,Sexual Harassment p-val,Sexual Harassment corr coeff
gaga make gag,0.070484,0.774597
make gag,0.070484,0.774597
run country,0.070484,-0.774597
gag lady,0.070484,0.774597
make gag lady,0.070484,0.774597


,Trolling p-val,Trolling corr coeff
run country,0.070484,0.774597
vote person,0.070484,0.774597
trump child,0.070484,0.774597
biden fraud,0.070484,0.774597
virus spread,0.070484,0.774597


,Others p-val,Others corr coeff
attack vienna,0.070484,0.774597
terror attack,0.070484,0.774597
biden obama,0.116117,0.707107
attack austria,0.116117,0.707107
civil war,0.124880,0.695608


,Toxic p-val,Toxic corr coeff
kamalas black dementia,0.070484,0.774597
joe 47 yrs,0.070484,0.774597
dogma corruption,0.070484,0.774597
dementia joe forgot,0.070484,0.774597
peoples world depend,0.070484,0.774597


In [ ]:
    corr_df = pd.DataFrame(columns = ['corr coeff', 'p-value'])
        dp = df[(df[label + " "] > 0)].sort_values(by=[label], ascending = True).head(10)
        top_features = list(dp[label].index)
        #print(dp[label])
        ds = df_2[label].T[top_features]
        #display(ds.T[top_features])
       
        corr_df['words'] = dp.index
        corr_df['p-value'] = list(dp[label])
        corr_df['corr coeff']  = list(ds)
        corr_df = corr_df.set_index('words')
        corr_df.index.name = fn + " - " + label
        display(corr_df)
        corr_df.to_csv(dest_folder+"CORR_ANALYSIS_"+file)
        

In [40]:
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]
src_folder =  "tfidf_scores_POS_per_video/"

for file in genre_files:
    df = pd.read_csv(src_folder+file, index_col = 0) 
    features = df.columns.difference(labels)
    for label in labels: 
        dl = df[df[label] == 1][features]
        dl = dl.T.sort_values(by=[dl.T.columns[0]], ascending = False).head(10)
        dl.index.name = file
        dl = dl.rename({dl.columns[0]: label}, axis='columns')
        
        display(dl)


,Direct Harassment
gaming_channels_GENRE.csv,
noun,5.932765
adj,5.304345
propn,4.375266
verb,3.595740
pron,3.011161
det,2.660375
aux,2.203011
det noun,2.009084
adj noun,1.897509


,Hate Speech
gaming_channels_GENRE.csv,
noun,33.668220
propn verb,27.484534
propn,20.021959
verb,18.332486
propn aux,5.975343
aux,4.283330
propn aux det noun,4.265662
noun noun,4.117543
propn aux det,3.887602


,Sexual Harassment
gaming_channels_GENRE.csv,
noun,1.829969
verb,1.301574
propn,1.190427
adj,1.015586
pron,0.907469
adj propn,0.868857
propn verb noun,0.708981
propn verb,0.655255
adp noun noun propn,0.613291


,Trolling
gaming_channels_GENRE.csv,
propn,4.380458
noun,3.728319
verb,3.490139
aux,3.217730
punct,3.142331
noun punct,2.980001
pron,2.825418
adv det propn noun,2.798669
punct adv det propn,2.798669


,Others
gaming_channels_GENRE.csv,
verb,0.981996
verb propn,0.855102
verb det propn,0.658518
adp det noun,0.636822
det,0.601597
propn,0.586069
adp det,0.581903
det propn,0.510162
adp,0.469519


...

,Hate Speech
news_channel_GENRE.csv,
propn,88.870788
noun,80.992302
punct,58.195538
verb,50.482515
adp,46.099317
propn propn,38.031241
det,36.809460
aux,32.976515
emoji,32.881169


,Sexual Harassment
news_channel_GENRE.csv,
propn,3.195031
noun,2.269076
verb,2.003061
propn propn,1.933734
det,1.604342
punct,1.587059
pron,1.462102
punct punct,1.451485
propn verb,1.344852


,Trolling
news_channel_GENRE.csv,
propn,56.056145
noun,45.845107
verb,36.601344
punct,34.140985
propn propn,26.214219
adp,25.786085
aux,22.724393
det,22.264011
adj,17.683020


,Others
news_channel_GENRE.csv,
propn,8.910536
noun,7.566555
verb,5.984656
punct,5.852794
det,5.235247
adp,4.734272
pron,4.188924
det noun,3.754812
aux,3.699306


,Toxic
news_channel_GENRE.csv,
propn,52.543534
noun,50.549503
verb,32.217801
punct,28.924005
propn propn,22.642132
adp,21.683983
det,19.829922
aux,16.542350
emoji,16.095242
